In [1]:
import qubic.toolchain as tc
import qubic.rpc_client as rc
import qubitconfig.qchip as qc
from distproc.hwconfig import FPGAConfig, load_channel_configs
import numpy as np

# Load Configs and Define Circuit

define FPGA config; this has timing information for the scheduler. For now it is fine to use the default config

In [2]:
fpga_config = FPGAConfig()

load channel configs (firmware channel mapping + configuration, see [Understanding Channel Configuration](https://gitlab.com/LBL-QubiC/software/-/wikis/Understanding-Channel-Configuration) for details), and QChip object, which contains calibrated gates + readout.


In [3]:
channel_configs = load_channel_configs('channel_config.json')
qchip = qc.QChip('qubitcfg.json')

As an alternative to the above, if you're using the chipcalibration repository, you can load all three configs like this:

In [5]:
import chipcalibration.config as cfg
chipname = 'X4Y2/sian' #this is a folder in the 'qchip' submodule of chipcalibration, containing the name of your chip
fpga_config, qchip, channel_config = cfg.load_configs(chipname)

# `branch_fproc` Instruction

The instruction used for **branching** looks like this:

```json
{'name': 'branch_fproc', 'alu_cond': <'le' or 'ge' or 'eq'>, 'cond_lhs': <var or ival>, 
'func_id': function_id, 'scope': <list_of_qubits_or_channels> 'true': [instruction_list], 'false': [instruction_list]}
```

Let's break this down:
 1. Data is **requested** from the FPROC according to the provided `func_id`. In the version of the gateware we're simulating, `func_id` just indicates the qubit whose measurement result we want. A list of available FPROC channels can be found in `fpga_config.fproc_channels`
 2. Once the FPROC receives the request, it fetches the result of the **most recent previous measurement** on that channel and sends it to the **core(s)** that requested it
 3. The **core** makes a **branching decision** according to `cond_lhs <alu_cond> fproc_result`. For example, you can check if the measurement was 0 using: `cond_lhs = 0`, `alu_cond = 'eq'`, which implements: `0 == fproc_result`.
 4. If the expression evaluates to **True**, the block of instructions in the `true` field are executed, **else** the block in `false` is executed.

## A note about state classification/thresholding

The FPROC classsifies states by thresholding across the y-axis; any accumulated value with `x>0` gets classified to 0; `x<0` goes to 1. Future FPROC implementations will include custom thresholds and qutrit states.

![image](./reset_image.svg)

## FPROC Channels

A list of available FPROC channels in the current gateware can be found in `fpga_config.fproc_channels`:

In [8]:
fpga_config.fproc_channels

{'Q0.meas': FPROCChannel(id=('Q0.rdlo', 'core_ind'), hold_after_chans=['Q0.rdlo'], hold_nclks=64),
 'Q1.meas': FPROCChannel(id=('Q1.rdlo', 'core_ind'), hold_after_chans=['Q1.rdlo'], hold_nclks=64),
 'Q2.meas': FPROCChannel(id=('Q2.rdlo', 'core_ind'), hold_after_chans=['Q2.rdlo'], hold_nclks=64),
 'Q3.meas': FPROCChannel(id=('Q3.rdlo', 'core_ind'), hold_after_chans=['Q3.rdlo'], hold_nclks=64),
 'Q4.meas': FPROCChannel(id=('Q4.rdlo', 'core_ind'), hold_after_chans=['Q4.rdlo'], hold_nclks=64),
 'Q5.meas': FPROCChannel(id=('Q5.rdlo', 'core_ind'), hold_after_chans=['Q5.rdlo'], hold_nclks=64),
 'Q6.meas': FPROCChannel(id=('Q6.rdlo', 'core_ind'), hold_after_chans=['Q6.rdlo'], hold_nclks=64),
 'Q7.meas': FPROCChannel(id=('Q7.rdlo', 'core_ind'), hold_after_chans=['Q7.rdlo'], hold_nclks=64)}

Each named channel indexes a `FPROCChannel` object, which contains the physical channel `id` to use, as well as the additional timing paremeters.

The channel `id` is resolved by the assembler using the `channel_configs` object; e.g. for `Q0.meas`, the physical channel ID is given by `channel_configs['Q0.rdlo'].core_ind`.

`hold_after_chans` and `hold_nclks` control how much delay is added before executing the branch instruction. More specifically, the compiler tools will ensure that there are at least `hold_nclks` clock cycles between the end of the most recent pulse on the channels in `hold_after_chans` and the execution of the branch (or `read_fproc`) instruction. This is to give the FPGA enough time to process the measurement result and store it in the FPROC memory. Delays are added to the program using `idle` instructions, which halt processor execution until the specified timestamp (which is referenced to the same counter as the pulse `start_time`).

In the current gateware, the feedback latency is given by:
64 (clocks between end of `rdlo` pulse and start of branch instruction) + 8 (clocks to execute branch instruction) + 3 (clocks to load next pulse) = 75 clocks (or 150 ns)

# Example: Conditional Bit-flip Circuit

This circuit applies a bit flip on `Q0` conditioned on the classical measurement result from `Q1`. Note that no delays need to be inserted between the `read` on `Q1` and the pulses applied to `Q0`; this is resolved by the compiler according to the `FPROCChannel` objects in the `FPGAConfig`.

In [6]:
circuit = [
    {'name': 'delay', 't': 400.e-6, 'qubit': ['Q0', 'Q1']},
    {'name': 'X90', 'qubit': ['Q1']},
    {'name': 'read', 'qubit': ['Q1']},
    {'name': 'branch_fproc', 'alu_cond': 'eq', 'cond_lhs': 1, 'func_id': 'Q1.meas', 'scope': ['Q0'],
                'true': [
                            {'name': 'X90', 'qubit': ['Q0']}, 
                            {'name': 'X90', 'qubit': ['Q0']},

                ],
                'false': []},
    {'name': 'barrier', 'qubit':['Q0', 'Q1']},
    {'name': 'read', 'qubit': ['Q0']},
    {'name': 'read', 'qubit': ['Q1']},

]

## Compile and Assemble

Compile the program. 

In [7]:
compiled_prog = tc.run_compile_stage(circuit, fpga_config, qchip)

Run the assembler to convert the above program into machine code that we can load onto the FPGA:

In [8]:
raw_asm = tc.run_assemble_stage(compiled_prog, channel_configs)

## Connect to Server and Run Circuit

Now that we've defined our circuit and compiled it to machine code, we can submit it to the ZCU216 and run it.

Instantiate the runner client: